In [1]:
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
              'width': 1000,
              'height': 1000,
              'scroll': True,
})

{'width': 1000, 'height': 1000, 'scroll': True}

# Interaction of Parsed Data, Hashes, and Deployed Contracts
### GE Cybersecurity Project
<hr/>

# Prototype of System
![Prototype](./1.png)

# High-Level Software Interactions
![Image](./2.png)

# Program Flowchart
<img src="./3.png" height=600>

## Set up Environment and Load the Contract from Memory
Our setup consists of:
- Importing all required packages
- Loading the environment variables from ```.env```
- Setting the paths for GE files
- Loading the deployed contract into memory

In [2]:
# web3.../deploy.py
import json
from web3 import Web3
from dotenv import load_dotenv, find_dotenv
import os
from dateutil.parser import parse

# load from .env file
load_dotenv(find_dotenv())

import hashlib

# Constants
LOG_TXT_PATH = 'logfiles/workstationLog.txt'
DEVICE_XML_PATH = 'logfiles/Device.xml'
MAKO_TCW_PATH = 'logfiles/makoTest2.tcw'

# Run module to load contract
%run ./loadContract.ipynb

## Query Blockchain and Log File to Compare Hashes
The following functions are used to generate the hash of the stored configuration file and comparing it with the hash that is stored on-chain. Here are the steps involved:
1. Query the on-chain hash of the specified log file
2. Generate the hash of the file that is on our local filesystem
3. Compare the two:
    - If they are the same, no action is taken
    - If they differ, a new block is appended to the chain with the updated hash and other metadata
        - Includes time configuration was changed and computer/user ID of person who changed it

In [3]:
def fileParser(file):
    # Gather pertinent info from log file
    computer_name = None
    config_pushed = False
    config_complete = None
    with open(file, 'r') as file:
        for line in file.readlines():
            # print(line)
            # Extract computer name
            if 'desktop' in line.lower() and '.' not in line:
                computer_name = line.split(' ')[-1][:-1]
            if 'starting publish' in line.lower():
                pass# print(line)
            if 'download complete' in line.lower():
                # print(line)
                config_pushed = True
                config_complete = parse(line[:line.index(',')])
    if computer_name and config_pushed:          
        # print('Computer name: ' + computer_name)
        # print('Configuration changed on: ' + str(config_complete))
        return computer_name, str(config_complete)

In [4]:
# Read file in chunks (future-proofing) and generate hash:
def hashGenerator(file, buffer_size = 65536):
    """
    This function reads a given file in chunks and generates a corresponding
    SHA256 hash.

    Parameters
    ----------
    file : type 'str'
        relative path to file to hash
    buffer_size : {65536, 'other'}, optional
        number of bytes to read, by default 65536

    Returns
    -------
    file_hash
        the SHA256 hash of the file provided

    Raises
    ------
    N/A
    """
    file_hash = hashlib.sha256()
    # Read file as binary
    with open(file, 'rb') as f:
        chunk = f.read(buffer_size)
        # Keep reading and updating hash as long as there is more data:
        while len(chunk) > 0:
            file_hash.update(chunk)
            chunk = f.read(buffer_size)
    return file_hash

In [5]:
# Function to update blockchain
def updateBlockChain(date, new_hash, computer_id, pvsTx):
    """
    This function updates the blockchain by calling the 'store' function of the
    smart contract.

    Parameters
    ----------
    new_hash : type 'str'
        hash of file obtained from hashGenerator function
    date : type 'str'
        date the configuration was changed
    computer_id : type 'str'
        ID of the computer/user. Found in the top line of the log file
    Returns
    -------
    None

    Raises
    ------
    N/A
    """
    print("Updating contract...")

    # Get latest transaction:
    nonce = w3.eth.getTransactionCount(
        my_address
    )  # gives our nonce - number of transactions
    # Store new value for hashNumber:
    store_transaction = hash_storage.functions.addHash(date, new_hash, computer_id, pvsTx).build_transaction(
        {
            "gasPrice": w3.eth.gas_price,
            "chainId": chain_id,
            "from": my_address,
            "nonce": nonce,
        }
    )
    signed_store_tx = w3.eth.account.sign_transaction(
        store_transaction, private_key=private_key
    )
    send_store_tx = w3.eth.send_raw_transaction(signed_store_tx.rawTransaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(send_store_tx)
    print("Updated!")
    print(
        "New value of hash: " + hash_storage.functions.retrieve().call()[0]
    )

Encryption/Decryption Functions

In [6]:
from cryptography.fernet import Fernet
def encrypt(paramToEncrypt):
    # Read the key
    with open('mykey.key', 'rb') as mykey:
        key = mykey.read()
    # Encrypt the value
    f = Fernet(key)
    encryptedParam = f.encrypt(bytes(paramToEncrypt, 'utf-8'))
    return encryptedParam

def decrypt(paramToDecrypt):
    # Read the key
    with open('mykey.key', 'rb') as mykey:
        key = mykey.read()
        
    f = Fernet(key)
    decryptedParam = f.decrypt(paramToDecrypt)
    return decryptedParam
    
    

In [7]:
# Query the hash stored on the blockchain
on_chain_hash = hash_storage.functions.retrieve().call()[0]
print('On-chain hash: {}'.format(on_chain_hash))
# Generate the hash of the log file
device_hash = '0x' + hashGenerator(DEVICE_XML_PATH).hexdigest()
print('Local hash: {}'.format(device_hash))
# compare the two - if different, update the blockchain!
if on_chain_hash != device_hash:
    # Gather metadata:
    computer_id, date_changed = fileParser(LOG_TXT_PATH)
    # Get previous tx hash
    previousTxHash = w3.eth.get_block('latest')['transactions'][0].hex()
    # Encrypt the metadata before updating the chain
    computer_id, date_changed = encrypt(computer_id), encrypt(date_changed)
    # Update blockchain
    updateBlockChain(date_changed, device_hash, computer_id, previousTxHash)
else:
    print('No change detected. Exiting program.')

On-chain hash: 
Local hash: 0xfcf9b277a29ca61e5661ff86a2ce053748b0ca0abf7c887a98c78ec84094a149
Updating contract...
Updated!
New value of hash: 0xfcf9b277a29ca61e5661ff86a2ce053748b0ca0abf7c887a98c78ec84094a149


## Demo - See all Previous Transactions on Our Contract and the Metadata Associated with Each

In [8]:
# ONLY RUN THIS FOR DEMO PURPOSES
computer_id, date_changed = fileParser(LOG_TXT_PATH)
# Get previous tx hash
previousTxHash = w3.eth.get_block('latest')['transactions'][0].hex()
# Encrypt the metadata before updating the chain
computer_id, date_changed = encrypt(computer_id), encrypt(date_changed)
# Update blockchain
updateBlockChain(date_changed, device_hash, computer_id, previousTxHash)

Updating contract...
Updated!
New value of hash: 0xfcf9b277a29ca61e5661ff86a2ce053748b0ca0abf7c887a98c78ec84094a149


In [9]:
pvsTx = hash_storage.functions.retrieve().call()[3]
history = []
while pvsTx:
    tx = w3.eth.get_transaction(pvsTx)
    try:
        obj, params = hash_storage.decode_function_input(tx['input'])
        display(params['_time'])
        pvsTx = params['_previousTx']
        history.append(params)
    except:
        pvsTx = False        

'gAAAAABjWYZFL5rlVEknC2dXO5i6PvVrS8IKsA8sCnYpjEl-y8F1OUDWByc9b_I9hjybmBLkk2on7WAxF4jsVDxKbN00XDDQw2ttoJ3Q6k-IuhvLu3Ch96M='

'gAAAAABjWYYp5bJ7K8ClqvWed9jr_rAhHN8ETRJSxCWwZYPVJHBWesc8EQa7W8zlCgO-x60QE02wg2ehTf5873bSzdreOt74ke1qLnRjZY0vcTiLfug9bRQ='

'gAAAAABjWYYo7oQ3DV-nzISD56g8R3yeY1QzsUYyZ5xF7dUbYvG1zuJDR_Z54uVUmjL6J6-ROOKckf3lpEMlkma7okN8wBKXx60HJjU75pQ8mgMBwC3FexA='

In [10]:
params

{'_time': 'gAAAAABjWYYo7oQ3DV-nzISD56g8R3yeY1QzsUYyZ5xF7dUbYvG1zuJDR_Z54uVUmjL6J6-ROOKckf3lpEMlkma7okN8wBKXx60HJjU75pQ8mgMBwC3FexA=',
 '_hashNumber': '0xfcf9b277a29ca61e5661ff86a2ce053748b0ca0abf7c887a98c78ec84094a149',
 '_userID': 'gAAAAABjWYYoXhXbUe28Fr-A5bDaYd87X74TMm3YnjyxpgyWDR5ASG0OXXiTi7OcRFtVEjovRIk9aRslAGTmqJrn_euFOcxJJg==',
 '_previousTx': '0xce0cff02e093dd500965a378566a937fd555da402642b8ce9f59095b79be9e86'}

In [11]:
import pandas as pd
df = pd.DataFrame(history)
df = df[['_time', '_userID', '_hashNumber', '_previousTx']]
df.rename(columns={'_time': 'Date', '_userID': 'User ID', '_hashNumber': 'Hash', '_previousTx': 'Previous Transaction'}, inplace=True)
# df.set_index('_time', inplace=True)
df

,Date,User ID,Hash,Previous Transaction
0,gAAAAABjWYZFL5rlVEknC2dXO5i6PvVrS8IKsA8sCnYpjE...,gAAAAABjWYZFRJuFjF4m-Y5T9M8AlL2UGj66dL37slrZl2...,0xfcf9b277a29ca61e5661ff86a2ce053748b0ca0abf7c...,0xb54bf86dbda4c2659e6c53c1e8b332293d77cc1e0a4f...
1,gAAAAABjWYYp5bJ7K8ClqvWed9jr_rAhHN8ETRJSxCWwZY...,gAAAAABjWYYpM1mWhkCNkAvidMNWzO1wi_2j4WvLKAJCj9...,0xfcf9b277a29ca61e5661ff86a2ce053748b0ca0abf7c...,0x18bbb8a002390245076f5a48aafeca1707b0a67d6051...
2,gAAAAABjWYYo7oQ3DV-nzISD56g8R3yeY1QzsUYyZ5xF7d...,gAAAAABjWYYoXhXbUe28Fr-A5bDaYd87X74TMm3Ynjyxpg...,0xfcf9b277a29ca61e5661ff86a2ce053748b0ca0abf7c...,0xce0cff02e093dd500965a378566a937fd555da402642...
